<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
!pip install pyLDAvis

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

try:
    from collections.abc import Callable 
except ImportError:
    from collections import Callable  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [21]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [22]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(1)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57


In [23]:
%%time

bug_report = bug_report.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 66.3 ms, sys: 4.98 ms, total: 71.3 ms
Wall time: 71.5 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [24]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 490 ms, sys: 4.89 ms, total: 495 ms
Wall time: 493 ms


In [25]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 19.9 s, sys: 109 ms, total: 20 s
Wall time: 23.4 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [26]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

CPU times: user 26.2 s, sys: 506 ms, total: 26.7 s
Wall time: 26.8 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [27]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 9.32 ms, sys: 1.03 ms, total: 10.4 ms
Wall time: 13.2 ms


,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [28]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

CPU times: user 18.8 ms, sys: 0 ns, total: 18.8 ms
Wall time: 19.5 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [29]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 500:
      break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [30]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

CPU times: user 2.27 s, sys: 141 ms, total: 2.41 s
Wall time: 2.42 s


[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1),
 (4543, 2),
 (4544, 1),
 (4545, 1),
 (4546, 1),
 (4547, 1)]

In [31]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. "actually" word seen 1 time.
2. "builders" word seen 2 time.
3. "change" word seen 5 time.
4. "comment" word seen 1 time.
5. "compare" word seen 1 time.
6. "complete" word seen 1 time.
7. "consider" word seen 1 time.
8. "consistency" word seen 1 time.
9. "contain" word seen 1 time.
10. "default" word seen 1 time.
11. "derive" word seen 1 time.
12. "document" word seen 1 time.
13. "drive" word seen 1 time.
14. "editor" word seen 3 time.
15. "effect" word seen 1 time.
16. "ensure" word seen 1 time.
17. "external" word seen 3 time.
18. "extremely" word seen 1 time.
19. "fail" word seen 1 time.
20. "inconsistent" word seen 1 time.
21. "internal" word seen 1 time.
22. "maintain" word seen 2 time.
23. "mandatory" word seen 1 time.
24. "miss" word seen 1 time.
25. "modify" word seen 1 time.
26. "move" word seen 1 time.
27. "navigator" word seen 2 time.
28. "network" word seen 1 time.
29. "occur" word seen 1 time.
30. "open" word seen 1 time.
31. "optimal" word seen 1 time.
32. "problem" wo

In [37]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

[(0,
  '0.029*"extension" + 0.027*"target" + 0.026*"warn" + 0.021*"command" + '
  '0.020*"property" + 0.016*"plugin" + 0.015*"define" + 0.015*"element" + '
  '0.014*"attribute" + 0.012*"variable"'),
 (1,
  '0.067*"thread" + 0.039*"method" + 0.029*"javathread" + 0.029*"daemon" + '
  '0.029*"libjvmdylib" + 0.024*"threadblocked" + 0.019*"object" + '
  '0.018*"native" + 0.018*"wait" + 0.018*"libsystembdylib"'),
 (2,
  '0.099*"message" + 0.086*"bundle" + 0.051*"miss" + 0.041*"resolve" + '
  '0.036*"orgeclipseosgi" + 0.031*"dispose" + 0.030*"subentry" + '
  '0.024*"require" + 0.017*"method" + 0.012*"widget"'),
 (3,
  '0.067*"public" + 0.060*"import" + 0.042*"display" + 0.033*"return" + '
  '0.025*"string" + 0.021*"static" + 0.014*"listener" + 0.013*"private" + '
  '0.012*"swtnone" + 0.011*"snippet"'),
 (4,
  '0.022*"change" + 0.012*"support" + 0.010*"provide" + 0.010*"currently" + '
  '0.008*"return" + 0.008*"remove" + 0.008*"different" + 0.007*"comment" + '
  '0.007*"current" + 0.007*"creat

In [38]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.3510130564382505

Coherence Score:  0.5965746031679003
CPU times: user 36.1 s, sys: 603 ms, total: 36.7 s
Wall time: 36.1 s


In [39]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

CPU times: user 17.8 s, sys: 554 ms, total: 18.4 s
Wall time: 23.3 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.103152  0.027116       1        1  14.913460
6     -0.052749  0.170432       2        1  13.036987
7     -0.015728  0.142600       3        1  13.032634
9     -0.199150  0.026192       4        1  12.656514
5     -0.174550 -0.018412       5        1  10.486859
8     -0.138710 -0.089327       6        1  10.040825
3      0.089472 -0.347972       7        1   8.088975
0     -0.001038 -0.026664       8        1   6.924378
1      0.287501  0.048730       9        1   6.607642
2      0.308104  0.067307      10        1   4.211725, topic_info=                  Term          Freq         Total Category  logprob  loglift
32             project  20777.000000  20777.000000  Default  30.0000  30.0000
372             source  16288.000000  16288.000000  Default  29.0000  29.0000
589            message  12816.000000  12816.000000  Default  28.0000  28.0000
13              editor  12953.000000  12953.000000  Default  27.0000  27.0000
103            eclipse  29209.000000  29209.000000  Default  26.0000  26.0000
360             public   9539.000000   9539.000000  Default  25.0000  25.0000
101             dialog  11415.000000  11415.000000  Default  24.0000  24.0000
761             import  10231.000000  10231.000000  Default  23.0000  23.0000
1341            thread   7788.000000   7788.000000  Default  22.0000  22.0000
484             bundle   6441.000000   6441.000000  Default  21.0000  21.0000
116             method  19709.000000  19709.000000  Default  20.0000  20.0000
109             select  14590.000000  14590.000000  Default  19.0000  19.0000
52             display  10366.000000  10366.000000  Default  18.0000  18.0000
23                miss   4016.000000   4016.000000  Default  17.0000  17.0000
608             search   6078.000000   6078.000000  Default  16.0000  16.0000
448             return   7929.000000   7929.000000  Default  15.0000  15.0000
626        perspective   4876.000000   4876.000000  Default  14.0000  14.0000
130            feature   6225.000000   6225.000000  Default  13.0000  13.0000
358             plugin   8066.000000   8066.000000  Default  12.0000  12.0000
361            resolve   3089.000000   3089.000000  Default  11.0000  11.0000
1005         extension   3568.000000   3568.000000  Default  10.0000  10.0000
365             button   9097.000000   9097.000000  Default   9.0000   9.0000
2               change  15947.000000  15947.000000  Default   8.0000   8.0000
4365        javathread   3387.000000   3387.000000  Default   7.0000   7.0000
3241            daemon   3376.000000   3376.000000  Default   6.0000   6.0000
4346       libjvmdylib   3369.000000   3369.000000  Default   5.0000   5.0000
179             target   3298.000000   3298.000000  Default   4.0000   4.0000
466             action   5155.000000   5155.000000  Default   3.0000   3.0000
552               warn   3663.000000   3663.000000  Default   2.0000   2.0000
4290    orgeclipseosgi   2663.000000   2663.000000  Default   1.0000   1.0000
357        performance   1199.081443   1199.961390   Topic1  -5.3933   1.9022
2871          baseline    502.345949    503.225827   Topic1  -6.2633   1.9012
3197            jvmdll    429.877991    430.759017   Topic1  -6.4191   1.9009
342           approach    364.830423    365.710382   Topic1  -6.5831   1.9005
4258          backport    309.311344    310.192171   Topic1  -6.7482   1.9001
134             future    292.693487    293.573424   Topic1  -6.8034   1.8999
51             discuss    247.403992    248.283993   Topic1  -6.9715   1.8994
277          artifacts    220.727082    221.607088   Topic1  -7.0856   1.8989
118           practice    209.183817    210.063788   Topic1  -7.1393   1.8987
136    implementations    200.851834    201.731770   Topic1  -7.1800   1.8985
1213           iterate    192.148798    193.028930   Topic1  -7.2243   1.8983
926            promote    190.139